In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from PIL import Image
import os

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
 #       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
#pip install tensorflow 

In [4]:
import tensorflow as tf
def _float_feature(value):
    return tf.train.Feature(float_list = tf.train.FloatList(value = [value]))
def _byte_feature(value):
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list = tf.train.BytesList(value=[value]))
def _int_64_feature(value):
    return tf.train.Feature(int64_list = tf.train.Int64List(value = [value]))
def serialize_array(value):
    return tf.io.serialize_tensor(value)


In [5]:
def makeEx(image_filename, label):
    image = np.asarray(Image.open(image_filename).resize((400,400)))
    data = {
        "height": _int_64_feature(image.shape[0]),
        "width" : _int_64_feature(image.shape[1]),
        "channels": _int_64_feature(image.shape[2]),
        "label": _byte_feature(bytes(label)),
        "image": _byte_feature(serialize_array(image)),
    }
    
    return tf.train.Example(features=tf.train.Features(feature=data))
def makeRecordFile(files, labels, filename):
    filename += ".tfrecords"
    writer = tf.io.TFRecordWriter(filename)
    count = 0
    for i in range(len(files)):
        example = makeEx(files[i], bytes(labels[i],'utf-8'))
        writer.write(example.SerializeToString())
        count += 1
    
    writer.close()
    print("Wrote {} to the record file".format(count))


In [6]:
from PIL import Image
import os
i = 0
os.chdir("/kaggle/input/flickr8kimagescaptions/flickr8k/images")
files = os.listdir(os.getcwd())
labels = []
with open("/kaggle/input/flickr8kimagescaptions/flickr8k/captions.txt", "r") as f:
    labels = f.read().split("\n")[1:]
#for file in files:
 #   np.asarray(Image.open(file))
  #  file.

In [7]:
labels = [i[i.find(".jpg")+5:] for i in labels]

In [8]:
#https://towardsdatascience.com/a-practical-guide-to-tfrecords-584536bc786c
labels
unique_characters = list(set("".join(labels)))
num_unique = len(unique_characters)
print("The unique characters are " + "".join(unique_characters) + " and number " + str(num_unique))

The unique characters are m.?fNAqP7WJFB4GnUo:RrypE;3!LwOT)z(-IgdXSKesb'xV2il,8hZ 6Da0cYjuQkHM#"&=51C9tv and number 77


In [9]:
makeRecordFile(files, labels, "/kaggle/working/hello")

Wrote 8091 to the record file


In [10]:
def parseEx(element):
    data = {
        "height": tf.io.FixedLenFeature([], dtype=tf.int64),
        "width": tf.io.FixedLenFeature([], dtype=tf.int64),
        "channels": tf.io.FixedLenFeature([], dtype=tf.int64),
        "label": tf.io.FixedLenFeature([], dtype=tf.string),
        "image": tf.io.FixedLenFeature([], dtype=tf.string),
    }
    ct = tf.io.parse_single_example(element, data)
    
    height = ct["height"]
    width = ct["width"]
    channels = ct["channels"]
    label = ct["label"]
    image = ct["image"]
    
    image = tf.io.parse_tensor(image, out_type = tf.uint8)
    image = tf.cast(image, dtype = tf.int64)
    image = tf.cast(image, dtype=tf.float32)
    image = tf.reshape(image, [height, width, channels])
    return (image, label)

def parseDst(filename):
    dst = tf.data.TFRecordDataset(filename)
    
    dst = dst.map(
        parseEx
    )
    return dst
dst = parseDst("/kaggle/working/hello.tfrecords")
    

In [11]:
import matplotlib.pyplot as plt

for s, p in dst.take(1):
    print(np.array(s))
    print(p)

[[[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]]

 [[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]]

 [[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]]

 ...

 [[ 29.  29.  29.]
  [ 10.  10.  10.]
  [  1.   1.   1.]
  ...
  [107. 107. 107.]
  [112. 112. 112.]
  [116. 116. 116.]]

 [[ 16.  16.  16.]
  [  1.   1.   1.]
  [  1.   1.   1.]
  ...
  [108. 108. 108.]
  [110. 110. 110.]
  [112. 112. 112.]]

 [[  9.   9.   9.]
  [  0.   0.   0.]
  [  4.   4.   4.]
  ...
  [110. 110. 110.]
  [109. 109. 109.]
  [109. 109. 109.]]]
tf.Tensor(b'A child in a pink dress is climbing up a set of stairs in an entry way .', shape=(), dtype=string)


In [12]:
import keras
from keras.applications.vgg16 import VGG16

In [13]:
# copy the weights and configurations for the pre-trained models (only required since Kaggle Kernels are offline)
#!mkdir ~/.keras
#!mkdir ~/.keras/models
#!cp ../input/keras-pretrained-models/*notop* ~/.keras/models/
#!cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/

In [14]:
enc = VGG16(include_top=False, weights='/kaggle/input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', input_shape=(400,400,3))

In [15]:
enc.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 400, 400, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 400, 400, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 400, 400, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 200, 200, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 200, 200, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 200, 200, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 100, 100, 128)     0     

In [16]:
for enc_layer in enc.layers:
    enc_layer.trainable = False

In [17]:
#https://stackoverflow.com/questions/71191514/typeerror-inputs-to-a-layer-should-be-tensors-got-tensorflow-python-keras-en
from keras.layers import GRU, LSTM, Dense, TimeDistributed

##inputs = tf.keras.Input(shape=inputs.get_output_at(0).get_shape())(enc.output)
#inputs = tf.keras.layers.Flatten()(inputs)
x = tf.reshape(enc.output, (12, 12, 512))
x = LSTM(64, return_sequences=True)(x)

x = LSTM(32, return_sequences=True)(x)
x = TimeDistributed(Dense(100, activation="relu"))(x)
outputs = Dense(77, activation="softmax")(x)

model = keras.Model(inputs = enc.input, outputs = outputs)

In [18]:
#enc.layers[0].get_shape()

In [19]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [20]:
#model.fit(dst, epochs=10)

In [21]:
#def hello(image, label):
    #image = tf.reshape(image, shape = (image.shape + [1]))
    #return (image, label)
#dst.map(hello)

In [22]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 400, 400, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 400, 400, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 400, 400, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 200, 200, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 200, 200, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 200, 200, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 100, 100, 128)     0     

In [23]:
model.layers[0](dst)

<_MapDataset element_spec=(TensorSpec(shape=(None, None, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>